<div class='heading'>
    <div style='float:left;'><h1>CPSC 4300/6300: Applied Data Science</h1></div>
    <img style="float: right; padding-right: 10px; width: 65px" src="https://bsethwalker.github.io/assets/img/clemson_paw.png"> </div>

## Week 6 | Lab: Outlier Detection, Model Selection and Cross Validation

**Clemson University**<br>
**Instructor(s):** Tim Ransom<br>

---

## Learning Goals

By the end of this lab, you should be able to:
* Feel comfortable with splitting the training and validation sets
* Feel comfortable with model selection
* Feel comfortable with selecting the best model out of selected ones

-----------

In [ ]:
""" RUN THIS CELL TO GET THE RIGHT FORMATTING """
import requests
from IPython.core.display import HTML
css_file = 'https://raw.githubusercontent.com/bsethwalker/clemson-cs4300/main/css/cpsc6300.css'
styles = requests.get(css_file).text
HTML(styles)

In [ ]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
from pandas.plotting import scatter_matrix
import seaborn as sns
from scipy import stats
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import learning_curve
from sklearn.decomposition import PCA
from sklearn.inspection import permutation_importance
import warnings
warnings.filterwarnings('ignore')

# 1. Outlier Detection

`Outiler`: Outliers are values at the extreme ends of a dataset.

- Some outliers represent true values from natural variation in the population. Other outliers may result from incorrect data entry, equipment malfunctions, or other measurement errors.

- An outlier isn’t always a form of dirty or incorrect data, so you have to be careful with them in data cleansing. What you should do with an outlier depends on its most likely cause.

- There are different ways to detect Outliers. One of the common technique is `Interquartile Range`.

    - Each dataset can be divided into quartiles. 
    - The first quartile point indicates that 25% of the data points are below that value whereas second quartile is considered as median point of the dataset. 
    - The inter quartile method finds the outliers on numerical datasets by following the procedure below

         - Find the first quartile, Q1.
         - Find the third quartile, Q3.
         - Calculate the IQR. IQR= Q3-Q1.

- Define the normal data range with lower limit as Q1 – 1.5 * IQR and upper limit as Q3 + 1.5 * IQR.

- Any data point outside this range is considered as outlier and should be removed for further analysis.

- The concept of quartiles and IQR can best be visualized from the boxplot. It has the minimum and maximum point defined as Q1 – 1.5 * IQR and Q3 + 1.5 * IQR respectively. Any point outside this range is outlier.



In [ ]:
# reading the dataset 
df=pd.read_csv('data/housing.csv')
df

In [ ]:
# Description of the dataset
print(df.describe())

In [ ]:
# This is the function to remove special characters.
def remove_sign(x,sign):
    if type(x) is str:
        x = float(x.replace(sign,'').replace(',',''))
    return x

In [ ]:
# See the initial distribution in boxplots.
df=df[['price','type']]
df=pd.DataFrame(df)

df.price = df.price.apply(remove_sign,sign='$')
sns.boxplot(y='price', x='type',data=df)
plt.xticks(rotation=90)
plt.ylabel('Price ($)')

In [ ]:
# Create function to implement IQR method.
#Inter Quartile Range

def remove_outlier_IQR(df):
    Q1=df.quantile(0.25)
    Q3=df.quantile(0.75)
    IQR=Q3-Q1
    df_final=df[~((df<(Q1-1.5*IQR)) | (df>(Q3+1.5*IQR)))]
    return df_final

def remove_outlier_ZScore(df):
    df_final=df[(np.abs(stats.zscore(df)) < 3)]
    return df_final

![IQR](https://editor.analyticsvidhya.com/uploads/15066IQR-Boxplot.png)

In [ ]:
len(df)

In [ ]:
len(remove_outlier_ZScore(df.price))

In [ ]:
len(remove_outlier_IQR(df.price))

In [ ]:
# Revisit the boxplot after outlier removal. The indices of the bad data points are determined and those are removed from the initial dataset.
# Creating the box plot 
df_outlier_removed=remove_outlier_IQR(df.price)
df_outlier_removed=pd.DataFrame(df_outlier_removed)
ind_diff=df.index.difference(df_outlier_removed.index)

for i in range(0, len(ind_diff),1):
    df_final=df.drop([ind_diff[i]])
    df=df_final
# Creating the box plot   
sns.boxplot(y='price', x='type',data=df_final)
plt.xticks(rotation=90)
plt.ylabel('Price ($)')

As seen in the boxplot, the majority of the outliers are removed.

Check number of outliers removed. The total number of outliers determined by this process is 18722.

In [ ]:
len(ind_diff)

# Notes:
IQR or Hampel method are very successful for extreme outliers with a single pattern whereas DBSCAN is a better choice if we have data of different patterns. Let’s say if we have a linear data as well as a circular data, DBSCAN will be able to differentiate the samples into different groups. 

# 2. Model Selection

<a class="anchor" id="third-bullet"></a>
Model selection is the task of selecting a model from among various candidates on the basis of performance criterion to choose the best one. In the context of learning, this may be the selection a statistical model from a set of candidate models, given data. The goal of model selection is to find a model that can accurately predict new, unseen data while avoiding overfitting, which occurs when a model is too complex and fits the training data too closely.


## 2.1 Model Selection with Wine Dataset
We first examine on Wine dataset which is a common one in Machine learning datsets.


In [ ]:
# Load the wine dataset and below function will print the description of the dataset
wine = load_wine()
print(wine.DESCR)


#### Steps for Model Selection
The wine dataset is a multiclass classification problem that involves predicting the type of wine (one of three possible classes) based on 13 numerical features. Here is a general outline of how you might approach model selection for this problem:

1. Define the problem: In this case, the problem is to classify the type of wine based on 13 numerical features. The performance metric we want to optimize is classification accuracy.

2. Select a set of candidate models: Choose a set of models that are appropriate for the problem and the data. For example, we might consider logistic regression, decision trees, random forests, support vector machines, and neural networks.

3. Split the data: Split the wine dataset into training, validation, and test sets. We might use a 60/20/20 split, for example.

4. Train and evaluate each model: Train each candidate model on the training set and evaluate its performance on the validation set using the chosen performance metric (classification accuracy). For example, we might use grid search to explore different hyperparameters for each model, such as the regularization strength for logistic regression or the number of trees in a random forest. We would then evaluate the performance of each model using k-fold cross-validation on the training set and choose the best hyperparameters based on the average validation accuracy.

5. Select the best model: Compare the performance of each model on the validation set and choose the model with the best performance. In this case, we might choose the random forest model with the best validation accuracy. The final model is then evaluated on the test set to estimate its generalization performance.

By following this process, we can ensure that we select the best model for the wine dataset and avoid overfitting to the training data.



## 2.2 Building models
In this part, we will select the models and build them.

The snippets of code below implement the above steps for the Model Selection.  Let's walk through the code.

Lets split the data as Training set and Valitadion set.


In [ ]:
wine.feature_names

In [ ]:
# Split the data into training, validation, and test sets
X_trainval, X_test, y_trainval, y_test = train_test_split(wine.data, wine.target, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.25, random_state=42)

Lets choose the candidate models for evaluation. Here we are going with Logistic Regression,
Decision Tree, Support Vector classification and Random Forest.

In [ ]:
# Define a list of candidate models
models = [
    ('logistic', LogisticRegression()),
    ('decision_tree', DecisionTreeClassifier()),
    ('random_forest', RandomForestClassifier()),
    ('svm', SVC())
]

In [ ]:
# Define a dictionary of hyperparameters for each model
params = {
    'logistic': {'C': [0.1, 1.0, 10.0]},
    'decision_tree': {'max_depth': [None, 3, 5]},
    'random_forest': {'n_estimators': [10, 50, 100], 'max_depth': [None, 3, 5]},
    'svm': {'C': [0.1, 1.0, 10.0], 'kernel': ['linear', 'rbf']},
    
}

In [ ]:
# Iterate over each model and perform grid search to find the best hyperparameters
best_model = None
best_accuracy = 0.0
for name, model in models:
    grid_search = GridSearchCV(model, params[name], cv=5)
    grid_search.fit(X_trainval, y_trainval)
    accuracy = grid_search.score(X_val, y_val)
    if accuracy > best_accuracy:
        best_model = grid_search.best_estimator_
        best_accuracy = accuracy
    print(f"{name}: validation accuracy = {accuracy}")

In [ ]:
# Evaluate the best model on the test set
test_accuracy = accuracy_score(y_test, best_model.predict(X_test))
print(f"Best model ({type(best_model).__name__}): test accuracy = {test_accuracy}")

This code defines a list of candidate models (logistic regression, decision tree, random forest, SVM, and neural network), as well as a dictionary of hyperparameters for each model. It then iterates over each model, performs grid search to find the best hyperparameters, and evaluates the performance of each model on the validation set. The best model is selected based on the highest validation accuracy, and the final model is evaluated on the test set to estimate its generalization performance.

<div class="Histogram"></div>
These histograms show the distribution of feature values for the first two features of the Wine dataset.

In [ ]:
# Plot histograms of feature values for the first two features
plt.hist(wine.data[:, 0], bins=20)
plt.xlabel('Feature 0')
plt.ylabel('Count')
plt.show()

plt.hist(wine.data[:, 1], bins=20)
plt.xlabel('Feature 1')
plt.ylabel('Count')
plt.show()


This scatter plot shows the relationship between two features of the Wine dataset, with different colors indicating the target classes.

<div class="exercise"><b>Question 1: Interpreting Feature Distributions</b> 
    The histograms above show the distribution of two features from the Wine dataset. One of the distributions appears approximately normal, while the other is skewed. What could be the reason for this skewness?</div>

1. The skewed distribution suggests that the feature has a natural asymmetry, possibly due to physical or chemical properties of the wine.
2. The skewness is caused by an error in data collection, and the dataset should be cleaned before further analysis.
3. The skewness occurs because the feature represents a categorical variable incorrectly encoded as numerical values.
4. The skewness is irrelevant and does not affect model selection or data preprocessing.


**Store your answer to a variable called `answer` in below code cell:**

Example code:
```python
answer = 4
```

In [ ]:
# your code here
raise NotImplementedError

In [ ]:
# Plot a scatter plot of two features
plt.scatter(wine.data[:, 0], wine.data[:, 1], c=wine.target)
plt.xlabel('Feature 0')
plt.ylabel('Feature 1')
plt.show()

Below code uses the pairplot function from the seaborn library to create a scatter plot matrix of all pairwise combinations of features in the Wine dataset. The hue parameter is set to the target variable, which is used to color-code the points based on their target class. The diag_kind parameter is set to 'hist', which plots histograms of each feature on the diagonal instead of scatter plots.

In [ ]:
# Pair wise scatter plot
import seaborn as sns
import pandas as pd

# Load the Wine dataset
wine = load_wine()

# Create a DataFrame from the Wine dataset
df = pd.DataFrame(data=wine.data, columns=wine.feature_names)
df['target'] = wine.target

# Create a pairwise scatter plot
sns.pairplot(data=df, hue='target', diag_kind='hist')
plt.show()


This confusion matrix shows the number of true positives, true negatives, false positives, and false negatives for each class of the Wine dataset, based on the predictions of the best model on the validation set. 

<div class="exercise"><b>Question 2: Understanding the Pairwise Scatter Plot</b> The pairwise scatter plot above visualizes relationships between all numerical features in the Wine dataset, with color coding based on wine type (target variable).

Which of the following statements best describes insights that can be gained from this plot?</div>

1. The scatter plots suggest that all features are independent and uncorrelated, making feature selection unnecessary.
2. Some features show clear separation between different wine classes, indicating that classification models can achieve good accuracy.
3. The histograms on the diagonal confirm that all features follow a perfect normal distribution.
4. The scatter plots indicate that the dataset has a significant number of missing values, which may impact model performance.

**Store your answer to a variable called `answer` in below code cell:**

Example code:
```python
answer = 4
```

In [ ]:
# your code here
raise NotImplementedError

In [ ]:
"""from sklearn.metrics import plot_confusion_matrix

# Plot a confusion matrix for the best model
plot_confusion_matrix(best_model, X_val, y_val)
plt.show()"""

from sklearn.metrics import ConfusionMatrixDisplay

# Plot a confusion matrix for the best model
ConfusionMatrixDisplay.from_estimator(best_model, X_val, y_val)


<div class="exercise"><b>Question 3: Interpreting the Confusion Matrix</b> The confusion matrix above represents the classification performance of the best model on the validation set of the Wine dataset.

Which of the following statements best describes the model's performance based on this confusion matrix?</div>

1. The model has a perfect classification accuracy, as there are no misclassified instances.
2. The model correctly classifies most samples, but some misclassifications exist, which can be reduced with further hyperparameter tuning.
3. The model struggles with classifying at least one class, as evident from a high number of misclassifications.
4. The model performs poorly overall, as indicated by the presence of many false positives and false negatives.

**Store your answer to a variable called `answer` in below code cell:**

Example code:
```python
answer = 4
```

In [ ]:
# your code here
raise NotImplementedError

In [ ]:
# Compute the correlation matrix
corr = np.corrcoef(wine.data.T)

# Plot a heatmap of the correlation matrix
sns.heatmap(corr, cmap='coolwarm', xticklabels=wine.feature_names,
            yticklabels=wine.feature_names, annot=True, fmt='.2f')
plt.show()


<div class="exercise"><b>Question 4: Interpreting the Correlation Heatmap</b> The heatmap above displays the correlation between numerical features in the Wine dataset. A correlation value close to 1.0 indicates a strong positive correlation, while a value close to -1.0 indicates a strong negative correlation.

Which of the following conclusions can be drawn from the correlation heatmap?</div>

1. Some features are highly correlated, which suggests that dimensionality reduction techniques like PCA could be beneficial.
2. All features have very weak correlations with each other, meaning each contributes independently to the classification task.
3. The heatmap suggests that the dataset has missing values, as indicated by empty or zero correlation values.
4. There is no relationship between any of the features, so feature selection is unnecessary.

**Store your answer to a variable called `answer` in below code cell:**

Example code:
```python
answer = 4
```

In [ ]:
# your code here
raise NotImplementedError

Below learning curve shows how the accuracy of the best model improves as the size of the training set increases, based on cross-validation.

In [ ]:
# Plot a learning curve for the best model

train_sizes, train_scores, val_scores = learning_curve(best_model, X_train, y_train,
                                                        train_sizes=np.linspace(0.1, 1.0, 10),
                                                        cv=5, scoring='accuracy')

plt.plot(train_sizes, np.mean(train_scores, axis=1), label='Training score')
plt.plot(train_sizes, np.mean(val_scores, axis=1), label='Validation score')
plt.xlabel('Training set size')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

Below scatter plot shows the projection of the Wine dataset onto the first two principal components, with different colors indicating the target classes.

In [ ]:
# Fit PCA to the Wine dataset
pca = PCA(n_components=2)
X_pca = pca.fit_transform(wine.data)

# Plot a scatter plot of the first two principal components
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=wine.target)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()


Below bar chart shows the feature importances of the best model based on permutation importance, which measures the decrease in accuracy when a feature is randomly permuted.

In [ ]:
# Compute the feature importances using permutation importance
result = permutation_importance(best_model, X_val, y_val, n_repeats=10, random_state=0)
importances = result.importances_mean

# Plot a bar chart of the feature importances
plt.bar(wine.feature_names, importances)
plt.xticks(rotation=90)
plt.xlabel('Feature')
plt.ylabel('Importance')
plt.show()

We loaded the wine dataset and then extracted the features (X) and target (y) from the dataset. Next, we use scikit-learn's SelectKBest class to select the 5 best features using the chi-squared statistical test (chi2). Finally, we transform the original feature matrix X into the K-best feature matrix X_kbest using the fit_transform method of the SelectKBest object.

Note that in this example, we set k=5, which means we're selecting the 5 best features. You can adjust this value to select a different number of features depending on your needs.

In [ ]:
from sklearn.datasets import load_wine
from sklearn.feature_selection import SelectKBest, chi2

# Load the wine dataset
wine = load_wine()

# Extract the features and target
X, y = wine.data, wine.target

# Use chi-squared statistical test to select the K-best features
selector = SelectKBest(chi2, k=5)
X_kbest = selector.fit_transform(X, y)

# Print the indices of the selected features
print(selector.get_support(indices=True))
wine.feature_names

# 3. Cross Validation
Now lets see some problems in cross validation

- `Cross-validation` is a technique used in machine learning to evaluate the performance of a model on an independent dataset.
- It involves partitioning the original dataset into multiple subsets, called folds, and training the model on a subset of the data while using the remaining data for validation. 
- This process is repeated multiple times, with each fold used once for validation and the other folds used for training.
- `Cross-validation` is a useful technique because it provides a more robust estimate of the model's performance than a single train-test split. 
- By using multiple splits of the data, cross-validation reduces the risk of overfitting or underfitting to a particular dataset.

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Load iris dataset
iris = load_iris()

# Set X and y variables
X = iris.data
y = iris.target

# Initialize Logistic Regression object
logreg = LogisticRegression()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)

# Fit the model on the training set
logreg.fit(X_train, y_train)

# Predict the labels of validation set
y_pred = logreg.predict(X_test)

acc = accuracy_score(y_test, y_pred)

print("Accuracy score:", acc)


# Lets Play with Iris dataset.

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Load iris dataset
iris = load_iris()

# Set X and y variables
X = iris.data
y = iris.target

# Define number of folds for cross-validation
n_folds = 5

# Initialize KFold object
kf = KFold(n_splits=n_folds, shuffle=True)

# Initialize Logistic Regression object
logreg = LogisticRegression()

# Initialize empty list to store accuracy scores
scores = []

# Loop through each fold and train the model
for train_idx, val_idx in kf.split(X):
    X_train, y_train = X[train_idx], y[train_idx]
    X_val, y_val = X[val_idx], y[val_idx]

    # Fit the model on the training set
    logreg.fit(X_train, y_train)

    # Predict the labels of validation set
    y_pred = logreg.predict(X_val)

    # Calculate the accuracy score and append to scores list
    acc = accuracy_score(y_val, y_pred)
    scores.append(acc)
    
# Calculate the average accuracy score
avg_score = sum(scores) / len(scores)

print("Cross-validation average accuracy score:", avg_score)


# Notes:
In this example, we load the iris dataset and split it into X and y variables. We then define the number of folds for cross-validation and initialize a KFold object. We also initialize a Logistic Regression object and an empty list to store accuracy scores.
We loop through each fold, split the data into training and validation sets, fit the logistic regression model on the training set, and predict the labels of the validation set. We then calculate the accuracy score and append it to the scores list.

After all the folds are processed, we calculate the average accuracy score by summing the scores and dividing by the number of folds. Finally, we print the average accuracy score.

In [ ]:
scores

In [ ]:
# Create a histogram of accuracy scores
plt.hist(scores, bins=10)
plt.title('Accuracy Scores')
plt.show()

The below code will create a line plot that shows how the cross-validation average accuracy score changes as the number of folds increases. The line plot will show if increasing the number of folds improves or decreases the model's performance.

In [ ]:
# Initialize empty list to store average scores
avg_scores = []

lower_fold_range = 2
upper_fold_range = 11

# Loop through different values of n_folds
for n_folds in range(lower_fold_range, upper_fold_range):
    kf = KFold(n_splits=n_folds, shuffle=True)
    scores = []
    for train_idx, val_idx in kf.split(X):
        X_train, y_train = X[train_idx], y[train_idx]
        X_val, y_val = X[val_idx], y[val_idx]
        logreg.fit(X_train, y_train)
        y_pred = logreg.predict(X_val)
        acc = accuracy_score(y_val, y_pred)
        scores.append(acc)
    avg_score = sum(scores) / len(scores)
    avg_scores.append(avg_score)

# Create a line plot of average scores
plt.plot(range(lower_fold_range, upper_fold_range), avg_scores)
plt.title('Cross-Validation Average Accuracy Scores')
plt.xlabel('Number of Folds')
plt.ylabel('Average Accuracy Score')
plt.show()

## 3.1 K Fold Cross Validation

k-Fold cross-validation is a technique that minimizes the disadvantages of the hold-out method. k-Fold introduces a new way of splitting the dataset which helps to overcome the “test only once bottleneck”.

The algorithm of the k-Fold technique:

1. Pick a number of folds – k. Usually, k is 5 or 10 but you can choose any number which is less than the dataset’s length.
2. Split the dataset into k equal (if possible) parts (they are called folds)
3. Choose k – 1 folds as the training set. The remaining fold will be the test set
4. Train the model on the training set. On each iteration of cross-validation, you must train a new model independently of the model trained on the previous iteration
5. Validate on the test set
6. Save the result of the validation
7. Repeat steps 3 – 6 k times. Each time use the remaining  fold as the test set. In the end, you should have validated the model on every fold that you have.
8. To get the final score average the results that you got on step 6.

Change the value of k in the cell below to see how it affects the validation score!

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Load iris dataset
iris = load_iris()

# Set X and y variables
X = iris.data
y = iris.target

# Define number of folds for cross-validation
k = 5

# Initialize KFold object
kf = KFold(n_splits=k, shuffle=True)

# Initialize Logistic Regression object
logreg = LogisticRegression()

# Initialize empty list to store accuracy scores
scores = []

# Loop through each fold and train the model
for train_idx, val_idx in kf.split(X):
    X_train, y_train = X[train_idx], y[train_idx]
    X_val, y_val = X[val_idx], y[val_idx]

    # Fit the model on the training set
    logreg.fit(X_train, y_train)

    # Predict the labels of validation set
    y_pred = logreg.predict(X_val)

    # Calculate the accuracy score and append to scores list
    acc = accuracy_score(y_val, y_pred)
    scores.append(acc)

# Calculate the average accuracy score
avg_score = sum(scores) / len(scores)

print("Cross-validation average accuracy score:", avg_score)


In the code above, we set k to 5 for 5-fold cross-validation. We then initialize a KFold object with n_splits parameter set to k and shuffle parameter set to True to shuffle the data before splitting it into folds.

We looped through each fold, split the data into training and validation sets, fit the logistic regression model on the training set, and predict the labels of the validation set. We then calculate the accuracy score and append it to the scores list.

After all the folds are processed, we calculate the average accuracy score by summing the scores list and dividing by the number of folds. Finally, we print the average accuracy score.

In [ ]:
from sklearn.model_selection import learning_curve

train_sizes, train_scores, val_scores = learning_curve(model, X, y, cv=10, n_jobs=-1)

train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
val_mean = np.mean(val_scores, axis=1)
val_std = np.std(val_scores, axis=1)

plt.plot(train_sizes, train_mean, label='Training score')
plt.plot(train_sizes, val_mean, label='Cross-validation score')

plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1)
plt.fill_between(train_sizes, val_mean - val_std, val_mean + val_std, alpha=0.1)

plt.xlabel('Number of training examples')
plt.ylabel('Accuracy')
plt.legend(loc='best')
plt.show()


The above code will plot the learning curves for the model. The learning curves plot the accuracy of the model on the training and validation sets as a function of the number of training examples. If the gap between the training and validation scores is large, it could indicate overfitting.

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X, y, cv=10)
plt.boxplot(scores)
plt.title('Cross-validation scores')
plt.show()


In [ ]:
from sklearn.model_selection import learning_curve

train_sizes, train_scores, test_scores = learning_curve(
    model, X, y, cv=kf, scoring='neg_mean_squared_error', train_sizes=np.linspace(0.1, 1.0, 10))

train_scores_mean = -np.mean(train_scores, axis=1)
test_scores_mean = -np.mean(test_scores, axis=1)

plt.plot(train_sizes, train_scores_mean, 'o-', label='Training score')
plt.plot(train_sizes, test_scores_mean, 'o-', label='Cross-validation score')
plt.legend(loc='best')
plt.xlabel('Training examples')
plt.ylabel('Negative mean squared error')
plt.title('Learning curves')
plt.show()


Learning curves: Learning curves show the training and validation scores for a model as a function of the size of the training set. By plotting learning curves for each fold of the cross-validation, you can get an idea of how the model's performance changes as the amount of training data increases.

<div class="exercise"><b>Question 5: Evaluating Cross-Validation and Learning Curves</b> The plots above from section 3. Cross Validation illustrate different aspects of cross-validation and learning curves for a logistic regression model trained on the Iris dataset.

Which of the following statements best describes insights gained from these visualizations?</div>

1. Cross-validation accuracy improves indefinitely as the number of folds increases, indicating that using the maximum number of folds always results in the best model.
2. The learning curves indicate a high bias model since the training and validation scores remain consistently low across all training set sizes.
3. The learning curves suggest that as more training data is added, the validation accuracy improves while the training accuracy slightly decreases, reducing overfitting.
4. The cross-validation boxplot indicates that model performance varies significantly across different folds, suggesting instability in the learning process.

**Store your answer to a variable called `answer` in below code cell:**

Example code:
```python
answer = 4
```

In [ ]:
# your code here
raise NotImplementedError

In [ ]:
fig, axs = plt.subplots(2, 5, figsize=(20, 8))
axs = axs.flatten()

for i, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    axs[i].scatter(y_test, y_pred - y_test)
    axs[i].axhline(y=0, color='black', linestyle='--')
    axs[i].set_title(f'Fold {i+1}')

plt.suptitle('Residual plots')
plt.tight_layout()
plt.show()


Residual plots: Residual plots show the difference between the predicted and actual values as a function of the actual values. By plotting residual plots for each fold of the cross-validation, you can get an idea of how well the model is fitting the data and whether there are any patterns in the residuals.

In [ ]:
from sklearn.model_selection import learning_curve

fig, ax = plt.subplots()

for i, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    train_sizes, train_scores, test_scores = learning_curve(model, X_train, y_train, cv=5)
    
    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    test_mean = np.mean(test_scores, axis=1)
    test_std = np.std(test_scores, axis=1)
    
    ax.plot(train_sizes, train_mean, label=f'Train Fold {i+1}')
    ax.plot(train_sizes, test_mean, label=f'Test Fold {i+1}')
    
ax.set_xlabel('Training examples')
ax.set_ylabel('Accuracy')
ax.legend()
plt.show()


Above code creates learning curves for each fold of the k-fold cross-validation. The learning curves can help to identify if the model is overfitting or underfitting for any particular fold.

These visualizations can help to diagnose issues with the model or the data and improve the performance of the model.

## 3.2 Leave-One-Out Cross Validation

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

# Load dataset
iris = load_iris()
X = iris.data
y = iris.target

loo = LeaveOneOut()

model = LogisticRegression()

accuracy_scores = []

for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))

print(f'Mean accuracy: {np.mean(accuracy_scores):.2f}')


# Notes:
In this example, we use the LeaveOneOut class to split the data into training and testing sets for each sample in the dataset. We then fit the logistic regression model on the training data and compute the accuracy score on the testing data. Finally, we calculate the mean accuracy score across all the samples in the dataset.

Note that LOOCV can be very computationally expensive, especially for larger datasets, since it involves fitting the model n times (where n is the number of samples in the dataset). Therefore, it is generally recommended to use k-fold cross-validation instead, unless the dataset is very small.

In [ ]:
# Load dataset
iris = load_iris()
X = iris.data
y = iris.target

loo = LeaveOneOut()

model = LogisticRegression()

accuracy_scores = []

for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))

# Plot accuracy scores for each sample
plt.plot(np.arange(len(accuracy_scores)), accuracy_scores)
plt.axhline(np.mean(accuracy_scores), color='r', linestyle='--', label='Mean accuracy')
plt.xlabel('Sample index')
plt.ylabel('Accuracy')
plt.title('Leave-One-Out Cross Validation for Iris Dataset')
plt.legend()
plt.show()


In the above example, we plot the accuracy score for each individual sample in the dataset, as well as a horizontal line representing the mean accuracy score across all the samples. This can be a useful way to visualize the performance of the model on each individual sample, and to detect any outliers or unusual patterns in the accuracy scores.

In [ ]:

from sklearn.metrics import confusion_matrix


# Load dataset
iris = load_iris()
X = iris.data
y = iris.target

loo = LeaveOneOut()

model = LogisticRegression()

y_true = []
y_pred = []

for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model.fit(X_train, y_train)
    y_true.append(y_test[0])
    y_pred.append(model.predict(X_test)[0])

# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Plot confusion matrix
fig, ax = plt.subplots()
im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
ax.figure.colorbar(im, ax=ax)
ax.set(xticks=np.arange(cm.shape[1]),
       yticks=np.arange(cm.shape[0]),
       xticklabels=iris.target_names, yticklabels=iris.target_names,
       xlabel='Predicted label', ylabel='True label',
       title='Confusion matrix for Iris dataset (LOOCV)')
plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
fmt = 'd'
thresh = cm.max() / 2.
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        ax.text(j, i, format(cm[i, j], fmt),
                ha="center", va="center",
                color="white" if cm[i, j] > thresh else "black")
fig.tight_layout()
plt.show()


In the above example, we computed a confusion matrix for the model's predictions on the Iris dataset, using LOOCV to evaluate the performance. We then plot the confusion matrix as a heatmap, with the true labels on the y-axis and the predicted labels on the x-axis. This can be a useful way to visualize the model's performance across different classes, and to identify any patterns of misclassification.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import confusion_matrix

# Generate a synthetic dataset
X, y = make_classification(n_samples=100, n_features=5, n_informative=3, random_state=42)

# Define the model and the Leave-One-Out Cross Validation object
model = LogisticRegression(solver='lbfgs', max_iter=1000)
loo = LeaveOneOut()

# Initialize the confusion matrix
cm = np.zeros((2, 2))

# Loop over the LOO splits and train/test the model
for train_idx, test_idx in loo.split(X):
    X_train, y_train = X[train_idx], y[train_idx]
    X_test, y_test = X[test_idx], y[test_idx]
    
    # Fit the model on the training data
    model.fit(X_train, y_train)
    
    # Make predictions on the test data
    y_pred = model.predict(X_test)
    
    # Update the confusion matrix
    cm[y_test, y_pred] += 1

# Compute the accuracy and display the confusion matrix
accuracy = np.sum(np.diag(cm)) / np.sum(cm)
print("Accuracy: {:.2f}%".format(accuracy * 100))
print("Confusion matrix:\n", cm)

# Visualize the confusion matrix
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.colorbar()
plt.title('Confusion matrix for Logistic Regression (LOOCV)')
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.xticks([0, 1], ['Negative', 'Positive'])
plt.yticks([0, 1], ['Negative', 'Positive'])
plt.show()


In the above example, we generated a synthetic dataset and train a logistic regression model on it, using LOOCV to evaluate the performance. For each split, we compute the confusion matrix (the number of true positives, true negatives, false positives, and false negatives), and we accumulate the results over all splits. We then display the overall accuracy and the confusion matrix as a heatmap. This can be a useful way to visualize the model's performance and to identify any patterns in the errors that the model is making.

<div class="exercise"><b>Question 6: Evaluating LOOCV Performance</b> The visualizations above show accuracy trends and confusion matrices for models trained using Leave-One-Out Cross-Validation (LOOCV) on the Iris dataset and a synthetic dataset.

Which of the following statements best describes insights gained from these visualizations?</div>

1. LOOCV produces high variance in accuracy scores due to training on nearly all data except one sample at a time, leading to occasional misclassifications.
2. The confusion matrices indicate that LOOCV is not suitable for small datasets, as it fails to produce meaningful classification results.
3. The LOOCV accuracy is significantly lower than k-fold cross-validation, suggesting it is not a reliable evaluation method.
4. The confusion matrices show that LOOCV guarantees perfect classification, as every sample is used for training except one at each iteration.

**Store your answer to a variable called `answer` in below code cell:**

Example code:
```python
answer = 4
```

In [ ]:
# your code here
raise NotImplementedError

# END